In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="3"
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
# Load a dataset (e.g., a sentiment analysis dataset)
# Replace 'imdb' with your desired dataset if using Hugging Face datasets
dataset = load_dataset("imdb")
# Load the RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-large")
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

# Apply tokenization to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Rename the 'label' column to 'labels' for compatibility with the model
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

# Set the format to PyTorch tensors
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


/home/gayeon43/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-11 16:45:46.388326: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Skipping import of cpp extensions due to incompatible torch version 2.8.0+cu128 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info


In [2]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
# Load the pre-trained RoBERTa model for sequence classification
# Adjust num_labels based on your task (e.g., 2 for binary classification)
model = RobertaForSequenceClassification.from_pretrained("roberta-large", num_labels=2)
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    # evaluation_strategy="epoch", # Evaluate at the end of each epoch
    save_strategy="epoch",       # Save model at the end of each epoch
    # load_best_model_as_init_checkpoint=True, # Load best model for evaluation
    metric_for_best_model="accuracy", # Specify metric for best model
)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"], # Using test set for evaluation during training
    compute_metrics=compute_metrics,
)

trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss
10,0.720500
20,0.694800
30,0.673600
40,0.719300
50,0.675800
60,0.677100
70,0.704400
80,0.602400
90,0.342300
100,0.303500


TrainOutput(global_step=9375, training_loss=0.6938740569305419, metrics={'train_runtime': 26213.4609, 'train_samples_per_second': 2.861, 'train_steps_per_second': 0.358, 'total_flos': 6.98948522496e+16, 'train_loss': 0.6938740569305419, 'epoch': 3.0})